In [0]:
%python
import time
import sys
sys.path.append("/Workspace/AUS vs US Music Taste/AUS-vs-US-Music-Taste-Data-Project/src")
import pandas as pd
from pathlib import Path
import artist_handling
import TrackInfo 
import importlib
import requests
import WikiGenre


WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
HEADERS = {
    "User-Agent": "ChartGenreResearch/1.0 (personal research)"
}


In [0]:
%python

input_dir = Path("/Workspace/AUS vs US Music Taste/AUS-vs-US-Music-Taste-Data-Project/Data/Silver/Aria_silver")
csv_files = sorted(input_dir.glob("*.csv"))
j = 2000
for csv_file in csv_files: 
    df = pd.read_csv(csv_file)
    for i, row in enumerate(df.itertuples(index=False)):
        try:
            genres = WikiGenre.find_wiki_genres(row.song_title, row.artist, verbose = True)
            df.at[i, "wiki_genre"] = str(genres)  # "[]" if none found
        except Exception as e:
            # keep going regardless of error 
            df.at[i, "wiki_genre"] = "[]"
            print("ERROR:", row.song_title, "-", row.artist, "-", type(e).__name__)
        time.sleep(1.5)

    df.to_csv(f'/Workspace/AUS vs US Music Taste/AUS-vs-US-Music-Taste-Data-Project/Data/Silver/Aria_silver_genre_fixed/Aria_{j}.csv', index=False)
    j+=1


In [0]:
input_dir = Path("/Workspace/AUS vs US Music Taste/AUS-vs-US-Music-Taste-Data-Project/Data/Silver/Billboard_silver")
csv_files = sorted(input_dir.glob("*.csv"))
j = 2000
for csv_file in csv_files: 
    df = pd.read_csv(csv_file)
    for i, row in enumerate(df.itertuples(index=False)):
        try:
            genres = WikiGenre.find_wiki_genres(row.song_title, row.artist, verbose = True)
            df.at[i, "wiki_genre"] = str(genres)  # "[]" if none found
        except Exception as e:
            # keep going regardless of error 
            df.at[i, "wiki_genre"] = "[]"
            print("ERROR:", row.song_title, "-", row.artist, "-", type(e).__name__)
        time.sleep(1.5)

    df.to_csv(f'/Workspace/AUS vs US Music Taste/AUS-vs-US-Music-Taste-Data-Project/Data/Silver/Billboard_silver_genre_fixed/Billboard_{j}.csv', index=False)
    j+=1

## Adding LastFM genre to empty wiki genre 

In [0]:
df = pd.read_csv('/Workspace/AUS vs US Music Taste/AUS-vs-US-Music-Taste-Data-Project/Data/Silver/Billboard_silver_genre_fixed/Billboard_2000.csv')


In [0]:
import ast

import ast
import numpy as np

def fill_wiki_genre(df):
    def parse_list(val):
        if val is None or (isinstance(val, float) and pd.isna(val)):
            return []
        if isinstance(val, list):
            return val
        if isinstance(val, str):
            try:
                parsed = ast.literal_eval(val)
                return parsed if isinstance(parsed, list) else []
            except Exception:
                return []
        if isinstance(val, np.ndarray):
            return list(val)
        return []

    df = df.copy()

    df["genre"] = df["genre"].apply(parse_list)
    df["wiki_genre"] = df["wiki_genre"].apply(parse_list)

    mask = (
        df["wiki_genre"].apply(len).eq(0)
        & df["genre"].apply(len).gt(0)
    )

    df.loc[mask, "wiki_genre"] = df.loc[mask, "genre"]

    return df

In [0]:
filled_df = fill_wiki_genre(df)
display(filled_df)

In [0]:
display(df)

In [0]:
def count_empty_wiki_genre(df):
    def is_empty(val):
        if val is None or (isinstance(val, float) and pd.isna(val)):
            return True
        if isinstance(val, list):
            return len(val) == 0
        if isinstance(val, str):
            try:
                parsed = ast.literal_eval(val)
                return not isinstance(parsed, list) or len(parsed) == 0
            except Exception:
                return True
        if isinstance(val, np.ndarray):
            return len(val) == 0
        return True

    return df["wiki_genre"].apply(is_empty).sum()

## Checking on how many rows have incomplete genres

In [0]:
empty_count_df = count_empty_wiki_genre(df)
print(empty_count_df)

In [0]:
empty_count_filled_df = count_empty_wiki_genre(filled_df)
print(empty_count_filled_df)

## Removing all instances of ' music' and '-music' from the wiki_genre column. So that all instances of genres like 'pop music' or 'electorinc-music' are now 'pop' and 'electronic respectively'

In [0]:
import re

# def strip_music_suffix(df):
#     pattern = re.compile(r"(?:\s+music|-music)$", re.IGNORECASE)

#     df = df.copy()
#     df["wiki_genre"] = df["wiki_genre"].apply(
#         lambda lst: [pattern.sub("", g).strip() for g in lst]
#     )
#     return df

_music_suffix = re.compile(r"(?:\s+music|-music)$", re.IGNORECASE)

def strip_music_suffix(df, col="wiki_genre"):
    def parse_to_list(val):
        if val is None or (isinstance(val, float) and pd.isna(val)):
            return []
        if isinstance(val, list):
            return val
        if isinstance(val, str):
            try:
                parsed = ast.literal_eval(val)
                return parsed if isinstance(parsed, list) else []
            except Exception:
                return []
        return []

    df = df.copy()
    df[col] = df[col].apply(parse_to_list)
    df[col] = df[col].apply(lambda lst: [_music_suffix.sub("", g).strip() for g in lst if isinstance(g, str)])
    return df

In [0]:
stripped_df = strip_music_suffix(filled_df)
display(stripped_df)


In [0]:
def dedupe_wiki_genre(df):
    df = df.copy()

    df["wiki_genre"] = df["wiki_genre"].apply(
        lambda genres: list(dict.fromkeys(genres))
        if isinstance(genres, list)
        else []
    )

    return df

In [0]:
clean_df = dedupe_wiki_genre(stripped_df)
display(clean_df)

## LIst of all distict genres in df


In [0]:

def get_all_distinct_wiki_genres(df):
    all_genres = set()

    for val in df["wiki_genre"]:
        if val is None or (isinstance(val, float) and pd.isna(val)):
            continue

        if isinstance(val, str):
            try:
                val = ast.literal_eval(val)
            except Exception:
                continue

        if isinstance(val, list):
            for g in val:
                if isinstance(g, str) and g.strip():
                    all_genres.add(g.strip())

    return all_genres

In [0]:
distinct_genres = get_all_distinct_wiki_genres(clean_df)
print(distinct_genres)


In [0]:

distinct_genres = set()
input_dirs = [Path("/Workspace/AUS vs US Music Taste/AUS-vs-US-Music-Taste-Data-Project/Data/Silver/Aria_silver_genre_fixed"),Path("/Workspace/AUS vs US Music Taste/AUS-vs-US-Music-Taste-Data-Project/Data/Silver/Billboard_silver_genre_fixed")]
for input_dir in input_dirs: 
    csv_files = sorted(input_dir.glob("*.csv"))
   
    for csv_file in csv_files: 
        df = pd.read_csv(csv_file)
        df = strip_music_suffix(df)
        genre_list = get_all_distinct_wiki_genres(df)

        distinct_genres.update(genre_list)

print(distinct_genres)

## Apply genre mapping


In [0]:
GENRE_MAPPING = {
# -------------------------
# POP
# -------------------------
"pop": "pop",
"art pop": "pop",
"progressive pop": "pop",
"electropop": "pop",
"teen pop": "pop",
"indie pop": "pop",
"synth-pop": "pop",
"power pop": "pop",
"pop rock": "pop",
"bubblegum": "pop",
"bedroom pop": "pop",
"chamber pop": "pop",
"baroque pop": "pop",
"orchestral pop": "pop",
"sophisti-pop": "pop",
"europop": "pop",
"norwegian pop": "pop",
"k-pop": "pop",
"pop soul": "pop",
"pop rap": "pop",
"popular": "pop",

# -------------------------
# ROCK
# -------------------------
"rock": "rock",
"rock and roll": "rock",
"alternative rock": "rock",
"hard rock": "rock",
"soft rock": "rock",
"punk rock": "rock",
"garage rock": "rock",
"glam rock": "rock",
"psychedelic rock": "rock",
"art rock": "rock",
"arena rock": "rock",
"yacht rock": "rock",
"rockabilly": "rock",
"nu metal": "rock",
"alternative metal": "rock",
"rap rock": "rock",
"blues rock": "rock",
"funk rock": "rock",
"southern rock": "rock",
"slacker rock": "rock",
"piano rock": "rock",
"comedy rock": "rock",
 

# -------------------------
# EMO/PUNK
# -------------------------
"emo": "emo/punk",
"pop-punk": "emo/punk",
"dance-punk": "emo/punk",
"skate punk": "emo/punk",
"grunge": "emo/punk",
"post-grunge": "emo/punk",
"post-punk": "emo/punk",
"post-punk revival": "emo/punk",
"gothic rock": "emo/punk",
"emo pop": "emo/punk",
"emo rap": "emo/punk",
"new wave": "emo/punk",
"rage": "emo/punk",

# -------------------------
# HIP-HOP
# -------------------------
"hip-hop": "hip-hop",
"hip hop culture": "hip-hop",
"australian hip-hop": "hip-hop",
"east coast hip-hop": "hip-hop",
"west coast hip-hop": "hip-hop",
"southern hip-hop": "hip-hop",
"alternative hip-hop": "hip-hop",
"conscious hip-hop": "hip-hop",
"gangsta rap": "hip-hop",
"cloud rap": "hip-hop",
"dirty rap": "hip-hop",
"progressive rap": "hip-hop",
"political hip-hop": "hip-hop",
"christian hip-hop": "hip-hop",
"hardcore hip-hop": "hip-hop",
"hip-hop soul": "hip-hop",
"hyphy": "hip-hop",
"go-go": "hip-hop",
"snap": "hip-hop",
"rapping": "hip-hop",


# -------------------------
# trap/drill
# -------------------------
"trap": "trap/drill",
"edm trap": "trap/drill",
"trap metal": "trap/drill",
"drill": "trap/drill",
"uk drill": "trap/drill",
"brooklyn drill": "trap/drill",
"crunk": "trap/drill",
"crunk&b": "trap/drill",
"crunkcore": "trap/drill",

# -------------------------
# ELECTRONIC
# -------------------------
"electronic": "electronic",
"electronic dance": "electronic",
"electronic rock": "electronic",
"electronica": "electronic",
"techno": "electronic",
"uk garage": "electronic",
"2-step garage": "electronic",
"nu-disco": "electronic",
"electro": "electronic",
"electroclash": "electronic",
"electro-disco": "electronic",
"synthwave": "electronic",
"techno-pop": "electronic",
"hi-nrg": "electronic",
"trance": "electronic",
"eurotrance": "electronic",
"downtempo": "electronic",
"ambient": "electronic",
"chillwave": "electronic",
"future bass": "electronic",
"big beat": "electronic",
"drum and bass": "electronic",
"dubstep": "electronic",
"complextro": "electronic",
"moombahton": "electronic",
"moombahcore": "electronic",
"amapiano": "electronic",
"jersey club": "electronic",
"folktronica": "electronic",
"trip hop": "electronic",
"new age": "electronic",

# -------------------------
# DANCE
# -------------------------
"dance": "dance",
"dance-rock": "dance",
"dancehall pop": "dance",
"dance-pop": "dance",
"eurodisco": "dance",
"eurodance": "dance", 
"italo dance": "dance", 


# -------------------------
# HOUSE
# -------------------------
"tech house": "house",
"house": "house",
"deep house": "house",
"progressive house": "house",
"french house": "house",
"disco house": "house",
"future house": "house",
"leftfield house": "house",
"electro house": "house",
"vocal house": "house",
"tropical house": "house",
"eurohouse": "house",

# -------------------------
# DISCO/FUNK
# -------------------------
"disco": "disco/funk",
"funk": "disco/funk",
"disco-funk": "disco/funk",
"disco-pop": "disco/funk", 
"uk funky": "disco/funk", 
"post-disco": "disco/funk", 
"ska": "disco/funk", 
"italo disco": "disco/funk",

# -------------------------
# R&B
# -------------------------
"rhythm and blues": "r&b",
"contemporary r&b": "r&b",
"new jack swing": "r&b",
"quiet storm": "r&b",
"doo-wop": "r&b",
"urban contemporary": "r&b",
"urban contemporary gospel": "r&b",

# -------------------------
# SOUL
# -------------------------
"soul": "soul",
"neo soul": "soul",
"progressive soul": "soul",
"blue-eyed soul": "soul",
"slow jam": "soul",

# -------------------------
# COUNTRY
# -------------------------
"country": "country",
"country pop": "country",
"neotraditional country": "country",
"country folk": "country",
"country rap": "country",
"bluegrass": "country",
"country rock": "country",


# -------------------------
# FOLK
# -------------------------
"folk": "folk",
"traditional folk": "folk",
"folk rock": "folk",
"folk-pop": "folk",
"indie folk": "folk",
"western": "folk",

# -------------------------
# LATIN
# -------------------------
"latin": "latin",
"latin pop": "latin",
"latin rock": "latin",
"latin trap": "latin",
"salsa": "latin",
"reggaeton": "latin",
"mambo": "latin",
"tango": "latin",
"dominican dembow": "latin",

# -------------------------
# REGGAE / AFRO
# -------------------------
"reggae": "reggae/afro",
"reggae fusion": "reggae/afro",
"reggae rock": "reggae/afro",
"soca": "reggae/afro",
"ragga": "reggae/afro",
"afrobeat": "reggae/afro",
"afrobeats": "reggae/afro",
"pop reggae": "reggae/afro",
"dub": "reggae/afro",

# -------------------------
# JAZZ / BLUES
# -------------------------
"jazz": "jazz/blues",
"blues": "jazz/blues",
"boogie": "jazz/blues",

# -------------------------
# PSYCHEDELIC
# -------------------------
"psychedelic": "psychedelic", 
"psychedelic pop": "psychedelic", 
"psychedelic soul": "psychedelic", 
"psychedelic rap": "psychedelic", 

# -------------------------
# BALLAD
# -------------------------
"sentimental ballad": "ballad", 
"power ballad": "ballad", 
"ballad": "ballad", 
"ballade": "ballad", 

# -------------------------
# CLASSICAL
# -------------------------
"cinematic classical": "classical",

# -------------------------
# OTHER
# -------------------------
"soundtrack": "other",
"show tune": "other",
"musical film": "other",
"short film": "other",
"spoken word recording": "other",
"stage and screen": "other",
"crossover fiction": "other",
"parody": "other",
"novelty": "other",
"christmas": "other",
"comedy": "other",
"christian": "other",          
"contemporary christian": "other",
   

}

In [0]:
norm_distinct = {g.strip().lower() for g in distinct_genres}
missing = sorted(norm_distinct - set(GENRE_MAPPING.keys()))
print("Missing:", len(missing))
print(missing)

In [0]:
def apply_genre_mapping(df, col="wiki_genre", mapping=None, drop_empty=True):
    mapping = mapping or {}

    def map_list(genres):
        if not isinstance(genres, list):
            return []

        out = []
        seen = set()

        for g in genres:
            if not isinstance(g, str):
                continue

            key = g.strip().lower()
            mapped = mapping.get(key, g.strip())

            if drop_empty and (mapped is None or mapped == ""):
                continue

            if mapped not in seen:
                seen.add(mapped)
                out.append(mapped)

        return out

    df = df.copy()
    df[col] = df[col].apply(map_list)
    return df

In [0]:
temp_df = clean_df.copy()
display(temp_df)


In [0]:
temp_df = apply_genre_mapping(temp_df, mapping=GENRE_MAPPING)
display(temp_df)